In [43]:
import json
import pandas as pd
from tqdm.notebook import tqdm
from transformers import MarianTokenizer, MarianMTModel
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [44]:
langs = ['de', 'es', 'zh', 'it', 'ro', 'vi', 'ru', 'fr', 'cs', 'jap']

In [45]:
train.iloc[151]

id                                                           198
question       Which sovereign state has the highest speed li...
category                                                resource
type           ['dbo:Country', 'dbo:State', 'dbo:PopulatedPla...
question_de    Welcher souveräne Staat hat die höchste Geschw...
question_es    ¿Qué estado soberano tiene el límite de veloci...
Name: 151, dtype: object

In [37]:
train = pd.read_csv("../../data/at/dbpedia/task1_dbpedia_train_cleaned.csv", sep='$')

In [38]:
train.head()

,id,question,category,type
0,0,Was Jacqueline Kennedy Onassis a follower of M...,boolean,['boolean']
1,1,What is the name of the opera based on Twelfth...,resource,"['dbo:Opera', 'dbo:MusicalWork', 'dbo:Work']"
2,2,When did Lena Horne receive the Grammy Award f...,literal,['date']
3,3,Do Prince Harry and Prince William have the sa...,boolean,['boolean']
4,5,Which is the hierarchical BrainInfo ID of the ...,literal,['string']


In [ ]:
for lang in langs:
    model = MarianMTModel.from_pretrained(f'Helsinki-NLP/opus-mt-en-{lang}').to(device)
    tokenizer = MarianTokenizer.from_pretrained(f'Helsinki-NLP/opus-mt-en-{lang}')
    translation_list = list()
    
    for text in tqdm(train.question.values):
        # Tokenize the text
        batch = tokenizer([text], return_tensors="pt", padding=True).to(device)

        # Make sure that the tokenized text does not exceed the maximum
        # allowed size of 512
        batch["input_ids"] = batch["input_ids"][:, :512]
        batch["attention_mask"] = batch["attention_mask"][:, :512]
        # Perform the translation and decode the output
        translation = model.generate(**batch)
        translation_list.append(tokenizer.batch_decode(translation, skip_special_tokens=True)[0])
        
    train[f'question_{lang}'] = translation_list
    train.to_csv("../../data/at-dbpedia/dbpedia/task1_dbpedia_train_translated.csv", sep='$', index=False)

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

